In [183]:
#1-1

import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_1.csv')
#print(df.info())
#print(df.head())

df['총대출액'] = df['금액1'] + df['금액2']
dfgender = df.groupby(['year', 'gender', '지역코드'])[['총대출액']].sum().reset_index()
#print(dfgender)
df_pivot = dfgender.pivot_table(index = ['year', '지역코드'], columns = 'gender', values = '총대출액', fill_value = 0)
df_pivot['abs_diff'] = abs(df_pivot[0] - df_pivot[1])
result = df_pivot['abs_diff'].idxmax()
print(result[1])  #df_pivot이 이중인덱스 이므로 두번째 인덱스

#1-2
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_2.csv')
#print(df.info())
#print(df.head())

df1 = df[df['구분'] == '검거건수'].set_index('연도').drop('구분', axis=1)
df2 = df[df['구분'] == '발생건수'].set_index('연도').drop('구분', axis=1)
dfratio = df1 / df2
dfmax = dfratio.idxmax(axis =1)
dfmax_reset = dfmax.reset_index().rename({0:'범죄유형'}, axis=1)

df1_melt = df1.reset_index().melt(id_vars = '연도', var_name = '범죄유형', value_name = '검거건수')
dffinal = pd.merge(dfmax_reset, df1_melt, on=['연도', '범죄유형'], how='left')
total_sum = dffinal['검거건수'].sum()
print(total_sum)

#1-3
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_3.csv')
#print(df.info())
#print(df.head())

df['평균만족도'] = df['평균만족도'].fillna(df['평균만족도'].mean())
df1 = df.groupby(['부서', '등급'])['근속연수'].mean().apply(np.floor).reset_index().rename(columns = {'근속연수' : '평균근속연수'})
#print(df1)

#merge로 결측치 대치
df = df.merge(df1,on = ['부서', '등급'], how='left')
df['근속연수'] = np.where(
    df['근속연수'].isnull(),
    df['근속연수'] == df['평균근속연수'],
    df['근속연수']
)

#print(df.isna().sum())

cond1 = (df['부서'] == 'HR')
cond2 = (df['등급'] == 'A')
result1=df.loc[cond1 & cond2, '근속연수'].mean()

cond3 = (df['부서'] == 'Sales')
cond4 = (df['등급'] == 'B')
result2 = df.loc[cond3 & cond4, '근속연수'].mean()

print(result1 + result2)

143
9879
27.7


In [ ]:
#2

import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_test.csv')

#2-1 (인포)
#print(train.info())
#print(test.info())
#print(train.head())

#2-2 (독-종 분리)
test_id = test['ID']
train_X = train.drop('라벨', axis=1)
train_y = train['라벨']
test_X = test.drop('라벨', axis=1)
test_y = test['라벨']

#2-3 (cat-num 분리)
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = test_X.select_dtypes('number').columns.to_list()

#2-4 (검증 데이터 생성)
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size = 0.3, random_state=1)
#print(train_X.shape, valid_X.shape, train_y.shape, valid_y.shape)

#2-5 (전처리)
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
oe = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)
stdscaler = StandardScaler()

train_X_oe = oe.fit_transform(train_X[cat_columns])
valid_X_oe = oe.transform(valid_X[cat_columns])
test_X_oe = oe.transform(test_X[cat_columns])

train_X_std = stdscaler.fit_transform(train_X[num_columns])
valid_X_std = stdscaler.transform(valid_X[num_columns])
test_X_std = stdscaler.transform(test[num_columns])

train_X_preprocessed = np.concatenate([train_X_oe, train_X_std], axis=1)
valid_X_preprocessed = np.concatenate([valid_X_oe, valid_X_std], axis=1)
test_X_preprocessed = np.concatenate([test_X_oe, test_X_std], axis=1)

#2-6 (모델 적합)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(train_X_preprocessed, train_y)

#2-7 (검증 - 성능 확인)
from sklearn.metrics import f1_score
pred_val = rf.predict(valid_X_preprocessed)
print(f1_score(valid_y, pred_val, average = 'macro'))

#2-8 (최종 예측)
test_pred = rf.predict(test_X_preprocessed)
test_pred = pd.DataFrame(test_pred, columns=['pred'])

#2-9 (생성, 저장)
result = pd.concat([test_id, test_pred], axis=1)
result.to_csv('result.csv', index=False)
print(pd.read_csv('result.csv').tail(50))
print(pd.read_csv('result.csv').info())

0.3386279352563086
       ID  pred
670   960     0
671  2199     0
672   712     0
673  1753     0
674   556     0
675  1942     0
676  1235     0
677  2186     0
678   998     0
679  1916     0
680  2301     0
681  1791     0
682  1874     0
683   731     0
684  1644     0
685   120     0
686   155     0
687  2165     0
688  1040     0
689  1357     0
690  1902     0
691   185     0
692  1847     0
693  2317     0
694  1025     0
695   819     0
696  2018     0
697  1715     0
698  2370     0
699   919     0
700   236     0
701  2041     0
702  1289     0
703  2051     1
704    21     0
705   600     0
706  1639     0
707  1980     0
708   320     0
709  2194     0
710   477     0
711   178     0
712  1050     0
713   868     0
714  1832     0
715  1488     0
716   172     0
717   768     0
718  1866     0
719   786     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  -------------

In [ ]:
#3-1

import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_1.csv')
#print(df.info())
#print(df.head())

#3-1-1
import statsmodels.api as sm
import statsmodels.formula.api as smf
formula = 'design ~ tenure + f2 + f3+ f4+ f5'
model = smf.ols(formula, data=df).fit()
p_value = model.pvalues[1:]
non_sig_p_value = p_value[p_value>=0.05]
print(len(non_sig_p_value))

#3-1-2
from scipy.stats import pearsonr
train = df.loc[df['id'] <=140, :]
test = df.loc[df['id']>140, :]

train_X = train.drop('design', axis=1)
train_y = train['design']
test_X = test.drop('design', axis=1)
test_y = test['design']

pred_val = model.predict(train_X)
statistic, p_value = pearsonr(train_y, pred_val)
print(statistic)

#3-1-3
from sklearn.metrics import root_mean_squared_error

pred_val = model.predict(test_X)
print(root_mean_squared_error(test_y, pred_val))

#3-2(마지막)

import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_2.csv')
#print(df.info())
#print(df.head())

#3-2-1
formula = 'churn ~ col1+col2+Phone_Service+Tech_Insurance'
model = smf.logit(formula, data=df).fit()
#print(model.summary())
p_value = model.pvalues
#print(p_value)
print(p_value['col1'])

#3-2-2
odds = model.params['Phone_Service']
odds_ratio = np.exp(odds)
print(odds_ratio)

#3-2-3
X = df.drop('churn', axis=1)
pred_val = model.predict(X)
result = pred_val[pred_val >= 0.3]
print(len(result))

2
0.9142907100055837
4.247702478659376
Optimization terminated successfully.
         Current function value: 0.640721
         Iterations 5
1.2070608549904602e-07
1.8670723445425634
450
